# Procesamiento de bases de indicadores

Este notebook documenta el proceso para obtener las bases de indicadores.

* Asociar datos del censo con cartografía de manzanas
* Asignar ids de cuadrante y colonia a las manzanas
* Agregar manzanas a la base de datos (GeoPackage)
* Construcción de agregados de variables por unidades

In [ ]:
import pandas as pd
import geopandas as gpd
from criminologia_cdmx. covariables import *
import numpy as np
import requests
import xml.etree.ElementTree as ET
import os
from zipfile import ZipFile
import shutil
from pathlib import Path

In [ ]:
%load_ext autoreload
%autoreload 2

## Asociar manzanas y censo

### Leer y preprocesar manzanas

In [ ]:
manzanas = gpd.read_file("datos/descargas/manzanas_2020_cdmx.zip")
# Quitamos las columnas que forman la CVEGEO (las vamos a tener de los dtos del censo)
manzanas = manzanas.drop(columns=['CVE_ENT', 'CVE_MUN',	'CVE_LOC', 'CVE_AGEB', 'CVE_MZA'])
manzanas.head()

DriverError: '/vsizip/datos/descargas/manzanas_2020_cdmx.zip' does not exist in the file system, and is not recognized as a supported dataset name.

### Leer y preprocesar censo

In [ ]:
censo = pd.read_csv("datos/descargas/conjunto_de_datos_ageb_urbana_09_cpv2020.zip", 
                    dtype={"ENTIDAD":str, "MUN": str, "LOC":str, "AGEB":str, "MZA": str})
# Nos quedamos sólo con las filas que tienen datos de manzanas (no localidad, agebs, etc)
censo = censo.loc[censo['MZA'] != '000']
# Construimos la cvegeo de las manzanas
censo['CVEGEO'] = censo['ENTIDAD'] + censo['MUN'] + censo['LOC'] + censo['AGEB'] + censo['MZA']
censo.head()

### Asociar manzanas con censo

In [ ]:
# Asociamos con join izquierdo porque no nos interesan las manzanas sin datos
manzanas = manzanas.merge(censo, on='CVEGEO', how='left')
manzanas.head()

## Identificadores de colonia y cuadrante

In [ ]:
# Leemos las geometrías de colonias y cuadrantes
colonias = gpd.read_file("datos/criminologia_capas.gpkg", layer='colonias')
cuadrantes = gpd.read_file("datos/criminologia_capas.gpkg", layer='cuadrantes')
# Extraemos puntos al interior de las manzanas
puntos_manzanas = manzanas.loc[:, ['CVEGEO', 'geometry']]
puntos_manzanas['geometry'] = puntos_manzanas['geometry'].representative_point()
# Unimos los puntos con las geometrías de cuadrantes y colonias
puntos_manzanas = (gpd
                   .sjoin(puntos_manzanas, colonias.to_crs(puntos_manzanas.crs)[['geometry', 'colonia_cve']], how='left')
                   .drop(columns='index_right'))
puntos_manzanas['colonia_cve'] = puntos_manzanas['colonia_cve'].astype('Int64')
puntos_manzanas = (gpd
                   .sjoin(puntos_manzanas, cuadrantes.to_crs(puntos_manzanas.crs)[['geometry', 'cuadrante_id']], how='left')
                   .drop(columns='index_right'))
# Unimos de regreso a las manzanas
manzanas = manzanas.merge(puntos_manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']], on='CVEGEO', how='left')
manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']]

Guardamos la capa con las relaciones

In [ ]:
manzanas[['CVEGEO','ENTIDAD', 'MUN', 'LOC', 'AGEB', 'MZA',
          'AMBITO', 'TIPOMZA', 'colonia_cve', 'cuadrante_id', 'geometry']].to_file("datos/descargas/manzanas_identificadores.gpkg", layer='manzanas', driver="GPKG")

## Tipos de datos

* Codificar bien los Nan
* Utilizar tipos de datos adecuados

In [ ]:
diccionario = get_diccionario_censo()
# Codificamos los Nan
manzanas = manzanas.replace('999999999', np.nan) 
manzanas = manzanas.replace('99999999', np.nan)
manzanas = manzanas.replace('*', np.nan)
manzanas = manzanas.replace('N/D', np.nan)
# Cambiamos los tipos de datos
campos_datos = diccionario['Nombre del Campo'].unique()
manzanas[campos_datos] = manzanas[campos_datos].astype('float')
manzanas.dtypes

### Guardarlos datos

El archivo es muy grande, vamos a guardar los datos por separado, en un archivo las geometrías de las manzanas y en otro las variables del censo

In [ ]:
manzanas.drop(columns=['geometry', 'ENTIDAD', 'NOM_ENT', 
                       'MUN', 'NOM_MUN', 'LOC', 'NOM_LOC', 
                       'AGEB', 'MZA']).to_csv("datos/censo_manzanas.zip", index=False)

La capa de manzanas la vamos a guardar sólo con la geometría y CVEGEO

In [ ]:
manzanas[['geometry', 'CVEGEO']].to_file("datos/descargas/covariables.gpkg", layer='manzanas', driver="GPKG")

## Uso de suelo

Dentro de las covariables también podemos usar datos de uso de suelo y estructura urbana, por lo pronto tenemos:

**Usos de Suelo**
* comercio
* industria
* servicios

**Estructura urbana**
* intensidad: la suma de los usos de suelo
* entropia: la mezcla

Todas estas variables salen del DENUE. Segúramente no son las ideales en este momento para el análisis delictivo, más adelante iremos incorporando otras.

Todos los conteos están calculados por manzana, pero para calcular mezcals de uso de suelo no es la mejor unidad porque es posible que se capture demasiado ruido, más adelante habría que hacer algún tipo de suavizado espacial.

In [ ]:
# Bajamos y leemos los datos
entropia_url = "https://www.dropbox.com/s/mke0xlxj832yrbz/MANZANAS_2020_ENTROPIA_MEXICO_PAIS.zip?dl=1"
r = requests.get(entropia_url, allow_redirects=True)
open("datos/descargas/" + 'entropia.zip', 'wb').write(r.content)
entropia = gpd.read_file("datos/descargas/entropia.zip")
entropia.head()

Extraemos sólo CDMX

In [ ]:
entropia = entropia.loc[entropia['CVEGEO'].str.slice(0,2) == '09']
entropia

Unimos a los identificadores de cuadrante y colonia. De los datos de arriba sólo queremos realmente los conteos `Sum_I`, `Sum_C` y `Sum_S`, las demás variables hay que recalcularlas.

In [ ]:
entropia = entropia[['CVEGEO', 'Sum_I', 'Sum_C', 'Sum_S']]
entropia = (manzanas[['CVEGEO', 'colonia_cve', 'cuadrante_id']]
            .merge(entropia, on='CVEGEO')
            .rename({'Sum_I':'Industria', 'Sum_C': 'Comercio', 'Sum_S':'Servicios'}, axis=1))
entropia

In [ ]:
entropia.to_csv("datos/usos_suelo.csv", index=False)

## Datos 911

Los datos de llamadas del 911 se encuentran disponibles en la página de datos abiertos de la CDMX. Hay un archivo por cada semestre desde 2019.

Por lo pronto, en lugar de proveer funciones como para carpetas y víctimas vamos a procesar un único archivo.

A continuación vamos a abrir un archivo csv que integra todos los concentrados semestrales sin ningún otro procesamiento.

In [ ]:
datos_911 = pd.read_csv("datos/descargas/911_19-21_unidas.csv.zip", dtype=object)
datos_911.head()

Lo primero que vamos a hacer es elimirar columnas repetidas y la columna geometry.

In [ ]:
datos_911 = datos_911.drop(columns=['aÃ±o_creacion', 'aÃ±o_cierre', 
                                    'aħo_creacion', 'aħo_cierre',
                                    'geometry'])
datos_911.head()

Ahora vamos a crear una columna de geometría a partir de las latitudes y longitudes, pero antes vamos a ver cuántos registros tenemos para ver cuántos perdemos en el proceso 

In [ ]:
print( "El número original de registros es %s"%(datos_911.shape[0]))

In [ ]:
datos_911.replace('NA', np.nan, inplace=True)
datos_911.dropna(subset=['longitud_centroide', 'latitud_centroide'], how='any', inplace=True)
print( "El número de registros limpios es %s"%(datos_911.shape[0]))

Parece que no perdimos ningún registro. Vamos a crear la columna de geometría.

In [ ]:
datos_911 = gpd.GeoDataFrame(datos_911, 
                             geometry=gpd.points_from_xy(datos_911.longitud_centroide, 
                                                         datos_911.latitud_centroide))
datos_911 = datos_911.set_crs(epsg=4326)
datos_911.plot()

In [ ]:
datos_911.to_file("datos/descargas/datos_911.gpkg", layer='incidentes',driver="GPKG")

De acuerdo a la documentación la columna manzana contiene el identficador de la manzana del incidente en las bases de INEGI. Unamos las bases de manzanas con las de incidentes para verificar

In [ ]:
manzanas = gpd.read_file("datos/descargas/manzanas_identificadores.gpkg")
# unidos = datos_911.merge(manzanas, left_on='manzana', right_on='CVEGEO')
# print( "Hay %s registros en la union "%(unidos.shape[0]))

In [ ]:
manzanas

Estamos perdiendo muchos registros, puede ser que no estén bien las claves (¿corresponden a diferentes años?).

Entonces mejor hagamos una unión por centroides.

In [ ]:
unidos = (datos_911.sjoin(manzanas[['colonia_cve', 
                                  'cuadrante_id', 'geometry']].to_crs(datos_911.crs))
         .drop(columns='index_right'))
print( "Hay %s registros en la union espacial"%(unidos.shape[0]))

De todas formas se pierden registros, pero muchos menos, entonces vamos mejor a usar esta forma de unir.

Hasta aquí ya tenemos la base del 911 con identificadores de manzanas, colonias y cuadrantes. Sólo falta entonces procesar las fechas

In [ ]:
unidos['fecha_creacion'] = pd.to_datetime(unidos['fecha_creacion'], dayfirst=True)
unidos['fecha_creacion']

Esta base ya está lista para usarse en la librería, la vamos a guardar comp pickle para recuperarla fácil

In [ ]:
unidos.to_pickle("datos/descargas/incidentes_911.pkl")

In [ ]:
unidos = pd.read_pickle("datos/descargas/incidentes_911.pkl")
type(unidos)

## Bases del DENUE

Para construir indicadores de uso de suelo vamos a incorporar las bases históricas del DENUE. El primer paso es descargarlas **todas** del sitio del INEGI. El sitio provee una herramienta para descrgar todos los datos usando Windows, para descargarlos desde Python, vamos a utilizar el XML que viene con dicha herramienta, este XML contiene las urls de todos los archivos que vamos a descargar, entonces el primer punto es _parsear_ el XML

In [ ]:
DOWNLOADS_PATH = "datos/descargas/denue/"
tree = ET.parse("datos/descargas/DescargaMasivaOD.xml")
root = tree.getroot()
datos = []
for archivo in root.iter('Archivo'):
    s = archivo.text
    tipo = s.rsplit("/")[-1].split(".")[-2].split("_")[-1]
    if tipo == 'shp' and (s.rsplit("/")[-2].split("_")[0] not in ['denue', 'esenciales']):
        url = archivo.text
        year = archivo.text.rsplit("/")[-2].split("_")[0]
        fname = s.rsplit("/")[-1]
        if os.path.exists(DOWNLOADS_PATH + fname):
            pass
        else:
            r = requests.get(url, allow_redirects=True)
            open(DOWNLOADS_PATH + fname, 'wb').write(r.content)
        try:
            gdf = gpd.read_file(DOWNLOADS_PATH + fname)
        except:
            zf = ZipFile('datos/descargas/denue/denue_00_11_0116_shp.zip')
            for f in zf.namelist():
                zinfo = zf.getinfo(f)
                if (zinfo.is_dir()):
                    if f.split("/")[-2] == 'conjunto_de_datos':
                        shps = [n for n in zf.namelist() if (n.startswith(f) and not n.endswith('/'))]
                        for s in shps:
                            basename = os.path.basename(s)
                            source = zf.open(s)
                            target = open(os.path.join('datos/descargas/denue/extracted/', basename), 'wb')
                            with source, target:
                                shutil.copyfileobj(source, target)
            gdf = gpd.read_file(os.path.join('datos/descargas/denue/extracted/', basename.split(".")[0] + '.shp'))
            [f.unlink() for f in Path('datos/descargas/denue/extracted/').glob("*") if f.is_file()] 
            
        campos = ['nom_estab', 'raz_social', 'codigo_act', 'nombre_act', 
                  'per_ocu', 'tipoCenCom', 'cve_ent', 'cve_mun', 'cve_loc', 
                  'ageb', 'manzana', 'geometry']
        gdf = gdf.loc[:,campos]
        gdf['cvegeo'] = gdf['cve_ent'] + gdf['cve_mun'] + gdf['cve_loc'] + gdf['ageb'] + gdf['manzana']
        gdf['year'] = year
        datos.append(gdf)
        
denue_total = pd.concat(datos)
denue_total.head()

In [ ]:
denue_total.loc[denue_total.cve_ent == '09'].to_file("datos/salidas/denue_cdmx.shp")

In [ ]:
denue_total.columns

In [ ]:
datos[0].plot()